<a href="https://colab.research.google.com/github/Maja-Thurup/Price-prediction-algorithm/blob/master/Price_Prediction_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [23]:
# physical_devices = tf.config.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

Let's load the data

In [24]:
df = pd.read_csv('final_cleaned_quotes - MainQuotes.csv')

At first glance I can tell that data has a lot of categorical features and requires some preprocessing.

I can't do exploratory analisys because of small amount of data. So I will:
- Do some preprocessing and data cleaning
- Prepare data to feed into neural network
- Build simple Neural network
- Run data throught NN to make sure it works

In [25]:
df.head(10)

,Estimate #,Vendors,Printed,Material,Micrometers,Configuration,Width (mm),Length (mm),Gusset (mm),Zipper,Features,SKUS,X qty,EXW/K,Notes,Unnamed: 15
0,EST-2360,Glen,Digital,PET/PE,110,3-Seal,76,248,0,No,None,2,1000,291,NaN,NaN
1,EST-2357,Sarah / Ella,Digital,PET/VMPET/PE,100,Stand Up Pouch,102,152,64,No,None,3,5000,82,Digital below $100/K,NaN
2,EST-2357,Sarah / Ella,Digital,PET/VMPET/PE,100,Stand Up Pouch,102,152,64,No,None,3,10000,"61,66666667",Digital below $100/K,NaN
3,EST-2356,Sarah / Ella,Plate,PET/VMPET/PE,100,Stand Up Pouch,184,254,90,Yes,None,4,5000,88,NaN,NaN
4,EST-2356,Sarah / Ella,Plate,PET/VMPET/PE,100,Stand Up Pouch,184,254,90,Yes,None,4,10000,78,NaN,NaN
5,EST-2356,Sarah / Ella,Plate,PET/VMPET/PE,100,Stand Up Pouch,184,254,90,Yes,None,4,25000,75,NaN,NaN
6,EST-2354,Glen,Plate,PET/AL/PE,110,3-Seal,102,127,0,No,None,1,5000,151,NaN,NaN
7,EST-2354,Glen,Plate,PET/AL/PE,110,3-Seal,102,127,0,No,None,1,20000,39,NaN,NaN
8,EST-2353,Glen,Plate,PET/VMPET/PE,110,Stand Up Pouch,178,222,76,Yes,None,3,5000,130,NaN,NaN
9,EST-2353,Glen,Plate,PET/VMPET/PE,110,Stand Up Pouch,178,222,76,Yes,None,3,10000,84,NaN,NaN


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Estimate #     211 non-null    object
 1   Vendors        211 non-null    object
 2   Printed        211 non-null    object
 3   Material       211 non-null    object
 4   Micrometers    211 non-null    int64 
 5   Configuration  211 non-null    object
 6   Width (mm)     211 non-null    object
 7   Length (mm)    211 non-null    object
 8   Gusset (mm)    211 non-null    object
 9   Zipper         211 non-null    object
 10  Features       211 non-null    object
 11  SKUS           211 non-null    int64 
 12  X qty          211 non-null    int64 
 13  EXW/K          211 non-null    object
 14  Notes          29 non-null     object
 15  Unnamed: 15    7 non-null      object
dtypes: int64(3), object(13)
memory usage: 26.5+ KB


In [27]:
df.apply(lambda x: x.nunique(), axis=0)

Estimate #       104
Vendors            6
Printed            3
Material          14
Micrometers        4
Configuration      4
Width (mm)        48
Length (mm)       45
Gusset (mm)       22
Zipper             5
Features           3
SKUS              11
X qty             17
EXW/K            158
Notes             20
Unnamed: 15        7
dtype: int64

*I am going to drop unnecesary columns* 

Also there two types of features: numerical and categorical

In [28]:
Features = df.drop(['Estimate #','Unnamed: 15','Notes'], axis=1 )

In [29]:
categorical = ['Vendors', 'Printed', 'Material', 
               'Configuration', 'Zipper', 'Features']

In [30]:
cat = Features.loc[:, categorical]
num = Features.drop(categorical, axis=1)

categorical before cleaning

In [31]:
cat.head(2)

,Vendors,Printed,Material,Configuration,Zipper,Features
0,Glen,Digital,PET/PE,3-Seal,No,None
1,Sarah / Ella,Digital,PET/VMPET/PE,Stand Up Pouch,No,None


In [32]:
target = num['EXW/K'].map(lambda x: x.replace('$','').replace(',','.'))
target = target.astype('float') # our target variables
num = num.drop('EXW/K', axis=1)

In [33]:
cat = pd.get_dummies(cat)

In [34]:
for col in ['Width (mm)', 'Length (mm)','Gusset (mm)']:
    num[col] = num[col].map(lambda x: x.replace(',','.')).astype('float')

Categorical after cleaning

In [35]:
cat.head(2)

,Vendors_Eric,Vendors_Glen,Vendors_Mars / Amber,Vendors_Sarah / Ella,Vendors_Sunny,Vendors_XWPAK,Printed_Digital,Printed_Not Printed,Printed_Plate,Material_KPET/PE,Material_MOPP/AL/PE,Material_MOPP/NYLON/VMPET/PE,Material_MOPP/PAPER/PE,Material_MOPP/PAPER/VMPET/PE,Material_MOPP/VMPET/PE,Material_NKR,Material_PAPER/PLA,Material_PET/AL/NY/PE,Material_PET/AL/PE,Material_PET/NY/PE,Material_PET/PE,Material_PET/VMPET/NYLON/PE,Material_PET/VMPET/PE,Configuration_2-Seal,Configuration_3-Seal,Configuration_8-Seal Pouch,Configuration_Stand Up Pouch,Zipper_CR Zipper,Zipper_No,Zipper_Pull Tab Zipper,Zipper_Tin-Tie,Zipper_Yes,Features_De-gassing Valve,Features_None,Features_Rounded Corners
0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0


Numerical after cleaning

In [36]:
num.head(2)

,Micrometers,Width (mm),Length (mm),Gusset (mm),SKUS,X qty
0,110,76.0,248.0,0.0,2,1000
1,100,102.0,152.0,64.0,3,5000


# Modelling

Here I am going to build simple Neural Network as an example how this dataset can be used to predict price

In [37]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense , Flatten, Activation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 


In [38]:
X = pd.concat([cat, num], axis=1).values #Features
y = target.values #targets

In [39]:
X.shape

(211, 41)

In [40]:
y.shape

(211,)

In [41]:
model = Sequential()

# The Input Layer :
model.add(Dense(32, kernel_initializer='normal',input_dim = X.shape[1], activation='relu'))

# The Hidden Layers :
model.add(Dense(16, kernel_initializer='normal',activation='relu'))
model.add(Dense(16, kernel_initializer='normal',activation='relu'))
model.add(Dense(16, kernel_initializer='normal',activation='relu'))

# The Output Layer :
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 32)                1344      
_________________________________________________________________
dense_6 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_7 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_8 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 17        
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.fit(X, y, epochs=10, batch_size=16, validation_split = 0.2)


Epoch 1/10
11/11 [==============================] - 1s 32ms/step - loss: 252.8919 - mean_absolute_error: 252.8919 - val_loss: 128.2395 - val_mean_absolute_error: 128.2395
Epoch 2/10
11/11 [==============================] - 0s 6ms/step - loss: 220.6912 - mean_absolute_error: 220.6912 - val_loss: 116.3954 - val_mean_absolute_error: 116.3954
Epoch 3/10
11/11 [==============================] - 0s 5ms/step - loss: 200.7050 - mean_absolute_error: 200.7050 - val_loss: 117.7396 - val_mean_absolute_error: 117.7396
Epoch 4/10
11/11 [==============================] - 0s 6ms/step - loss: 212.2138 - mean_absolute_error: 212.2138 - val_loss: 145.0186 - val_mean_absolute_error: 145.0186
Epoch 5/10
11/11 [==============================] - 0s 5ms/step - loss: 216.1080 - mean_absolute_error: 216.1080 - val_loss: 149.0675 - val_mean_absolute_error: 149.0675
Epoch 6/10
11/11 [==============================] - 0s 6ms/step - loss: 200.4777 - mean_absolute_error: 200.4777 - val_loss: 154.0701 - val_mean_abso

As we can see mean error is pretty high. It is possible to increase accuracy with:
- Larger number of samples
- Larger validation set
- More complicated Neural Network Architechture

As a conclusion I can say that NN will be a good solution for this kind of problem.